In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import warnings
from IPython.display import HTML
from IPython.display import IFrame

warnings.filterwarnings('ignore')
from bokeh.io import output_notebook, show, output_file, save
from bokeh.models import ColumnDataSource,FactorRange,Legend
from bokeh.palettes import Category20
from bokeh.plotting import figure
from bokeh.transform import factor_cmap,dodge
from bokeh.palettes import Spectral4
from bokeh.models import Panel
from bokeh.models.widgets import Tabs
import math
def moving_average(interval, window_size):
    window = np.ones(int(window_size)) / float(window_size)
    return np.convolve(interval, window, 'valid')

def gauss_filter(interval, sigma,scale):
    window = np.random.normal(0, sigma, 1000)
    return np.convolve(interval, window, 'valid')

# ___Exploration of regularity of 311-Service-Requests and the correlation with COVID-19___

***
## **Ⅰ. Motivation of our project**

### **Main purpose and value of the project**
*(1)Discovering regularity and features of 311-Service-Requests to provide crucial suggestions for different department of NYC, including suggestions under emergency circumstances such as COVID-19.*
*(2)Discovering the influence of COVID-19 on people's lives in NYC*

### **What is your dataset?**
*(1)311-Service-Requests of New York City (A complaint hotline of NYC)*
*(2)COVID-19 data of New York City*

### **Why did you choose this/these particular dataset(s)?**
*(1)311-Service-Requests of NYC provide up-to-date 311 complaint calls of different types in since 2010, which can illustrate regularity of various types and helps to generate valuable suggestions to different department of NYC*
*(2)COVID-19 dataset provide basic informations about cases, hospitalized and death amounts in NYC, enable us to combine the 311-Service-Request dataset with COVID-19 information and discover the influence of COVID-19 on people's lives in NYC*

### **What was your goal for the end user's experience?**
*(1)Get to know regularity, geographic feature and time feature of some important types of complaints, and help to generate crucial suggestions for different department of NYC*
 *(2)Get to know the influence of COVID-19 on people's lives in NYC*

***
## **Ⅱ. Basic stats. Let's understand the dataset better**

### **Write about your choices in data cleaning and preprocessing**
### *The original 311-Service-Requests dataset is over 12GB, which is hard for us to process. In order to reduce the size, firstly we remove many irrelavent columns and focus on the time, location, departments and types. Secondly we also chose to focus on data since 2015.01.01, so that we can even focus on these 5 years and make parallel comparison. Finally we selected 20 most common complaint types and filtered the dataset once again. After these data filter steps, our dataset came to 2GB, which is far more convenient for us to process than the original dataset.*

### **Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis.**
### *(1) 311-service dataset*

In [4]:
# this module read in the filtered dataset, and then select those data after 2015.01.01
df = pd.read_csv('focus_types.csv')
df['Created Date'] = pd.to_datetime(df['Created Date'], format = '%Y-%m-%d')
df['Closed Date'] = pd.to_datetime(df['Closed Date'], format = '%Y-%m-%d')
df =df.loc[df['Created Date'].dt.date >= datetime.date(2015,1,1)]
df['Count'] = np.ones([len(df),1])

In [8]:
# an overview of the dataset
df

,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Location,Count
4441018,2015-01-01 00:00:00,2015-01-07,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,"(40.8233716323884, -73.92738313721195)",1.0
4441019,2015-01-01 00:00:00,2015-01-07,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,"(40.85350505305637, -73.89441961846626)",1.0
4441020,2015-01-01 00:00:00,2015-01-05,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,"(40.8666990148464, -73.92648618310697)",1.0
4441021,2015-01-01 00:00:00,2015-01-03,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,"(40.61041295270483, -73.95150789023555)",1.0
4441022,2015-01-01 00:00:00,2015-01-05,HPD,Department of Housing Preservation and Develop...,HEAT/HOT WATER,"(40.599413792468944, -74.00168527686257)",1.0
...,...,...,...,...,...,...,...
13155721,2020-05-10 01:57:12,NaT,NYPD,New York City Police Department,Illegal Parking,"(40.70016226779197, -73.74174818167702)",1.0
13155722,2020-05-10 01:57:40,NaT,NYPD,New York City Police Department,Noise - Residential,"(40.864189772880415, -73.90335428261925)",1.0
13155723,2020-05-10 01:58:33,NaT,NYPD,New York City Police Department,Noise - Street/Sidewalk,"(40.864787270260365, -73.9023483356522)",1.0
13155724,2020-05-10 01:59:05,NaT,NYPD,New York City Police Department,Noise - Residential,"(40.641081574441856, -74.0144923423146)",1.0


In [10]:
# this module shows the focused types of complaint
Type = df.groupby('Complaint Type').sum().index
print('Complaint Types:', Type)

Complaint Types: Index(['Blocked Driveway', 'Building/Use', 'Derelict Vehicles',
       'Dirty Conditions', 'General Construction/Plumbing', 'HEAT/HOT WATER',
       'Illegal Parking', 'Noise', 'Noise - Commercial', 'Noise - Residential',
       'Noise - Street/Sidewalk', 'Noise - Vehicle', 'PAINT/PLASTER',
       'Request Large Bulky Item Collection', 'Rodent', 'Sewer',
       'Sidewalk Condition', 'Street Condition', 'Street Light Condition',
       'Traffic Signal Condition', 'UNSANITARY CONDITION', 'Water System'],
      dtype='object', name='Complaint Type')


In [17]:
# The code below can create and save the interactive plot named 'total_explainer.html', but it's quite time consuming, so we choose to read the saved file so that you don't have to run to see the results. 

#output_notebook(hide_banner=True)
#daterange = list(map(str,np.arange(0,367)))
##print(daterange)
##color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
#color = ['#1f77b4','#17becf','#ff7f0e','#2ca02c','#9467bd']
#p = figure(title="Amount of Daily Total Complaint",#x_axis_type='datetime',#x_range = FactorRange(factors=daterange),
#           plot_width=950, plot_height = 600,  x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')
##y_range=(0, 6500)
#for year, color in zip([2015, 2016,2017,2018,2019],color):#
#    #print(year)
#    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,1,1)) & (df['Created Date'].dt.date < datetime.date(year+1,1,1))]
#    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
#    group_temp = df_inter1.groupby('Created Date').sum()
#    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.01 — '+str(year+1)+'.01')#
#    del df_inter1,group_temp
#p.legend.location = "top_right"
#p.legend.click_policy="hide"
#p.xaxis.visible = True
#tab0 = Panel(child=p, title="All Types")
#output_file('total_explainer.html')
#show(p)
HTML(filename = 'total_explainer.html')

### *COVID-19 dataset*

In [18]:
# overview of daily cases count of COVID-19 in NYC
df_case = pd.read_csv('coronavirus-data-master/case-death.csv')
df_case

,DATE_OF_INTEREST,CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT
0,2/29/20,1,13,0
1,3/1/20,1,4,0
2,3/2/20,0,21,0
3,3/3/20,2,18,0
4,3/4/20,5,22,0
...,...,...,...,...
67,5/6/20,1372,125,95
68,5/7/20,1158,105,62
69,5/8/20,872,64,47
70,5/9/20,406,18,28


In [19]:
# overview of 'by-age' dataset
df_age = pd.read_csv('coronavirus-data-master/by-age.csv')
df_age

,AGE_GROUP,COVID_CASE_RATE,HOSPITALIZED_CASE_RATE,DEATH_RATE
0,0-17 years,266.66,21.66,0.58
1,18-44 years,1998.02,216.07,17.42
2,45-64 years,3267.00,806.45,162.84
3,65-74 years,3270.69,1564.54,530.38
4,75 and older years,3890.49,2473.06,1329.68
5,Citywide total,2186.78,579.92,177.74


In [20]:
# overview of 'by-sex' dataset
df_sex = pd.read_csv('coronavirus-data-master/by-sex.csv')
df_sex

,SEX_GROUP,COVID_CASE_RATE,HOSPITALIZED_CASE_RATE,DEATH_RATE
0,Female,2025.44,473.62,132.63
1,Male,2358.43,693.76,225.01
2,Citywide total,2186.78,579.92,177.74


In [21]:
# overview of 'borough' dataset
df_borough = pd.read_csv('coronavirus-data-master/boro.csv')
df_borough

,BOROUGH_GROUP,COVID_CASE_COUNT,COVID_CASE_RATE
0,The Bronx,41628,2829.56
1,Brooklyn,49865,1838.67
2,Manhattan,22646,1205.81
3,Queens,56719,2271.04
4,Staten Island,12714,2529.14
5,Citywide,183662,NaN


*** 
## **Ⅲ. Data Analysis**
### **Describe your data analysis and explain what you've learned about the dataset**
 *After filtering the 311 - service dataset contains only 7 different features. "Created time" and "Closed time" show the time-related variables, and these two can show if there is time rule for different complaint requests. Furthermore, it can be seen that some request has a very short time interval between the "Created time" and "Closed time", however, some show very long. So maybe it is related to a certain kind of request. Besides, the "Agency" and "Agency name" show which department is responsible for which request. The "Complaint type" will indicate the specific request which will be the main part of our analysis. The last feature is "Location", which store the coordinate of each request, and we can use this feature to compare the distribution of different types of request on the map.*
 
*The COVID - 19 dataset contains some features like "DATE_OF_INTEREST" which can be used to see how it related to time increase. Besides, the "AGE_GROUP", "SEX_GROUP" and "BOROUGH_GROUP" can show that how the COVID - 19 distribute agaists with those features. So it can be seen that the COVID - 19 maybe related to age sex and the location, also from the time feature we can also maybe find the trend of how it increase or decrease.*
### **If relevant, talk about your machine-learning**
 *We implement the KNN in our case to classify the 3 different types of request on the map. All the location of the request in the dataset will be as the X variables, and the type will be as y. So by using X to predict y, we will classify the certain point in the map as one of the three complaint request types.*

***
## **Ⅳ. Genre.** *Which genre of data story did you use?*
*The Film / Video / Animation  and annotated chart are the main genre that we use in our visualiztion*

*This is an example of the film genre of our visulization, the user can play or pause by push the button on bottom.*
### **Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?**
   * **Visual narrative** :
     - *Visual Structuring*: Consistent Visual Platform, Progress Bar 
     - *Highlighting*: Feature Distinction, Zooming
     - *Transition Guidance*:  Familiar objects
     
*Consistent visual platform and progress bar could help users explore information from different figures easily. Zooming could show more details especially in the map, which could help the user find latency patterns from different angles. For example, the heatmap shows a spatiotemporal pattern for four different complaint types. Zooming function could help the user find which street has a high happening rate for the complaint type 'Block Driveway'.* 

 ![Drag Racing](Heatmapeg.png)
  
### **Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why?**
   * **Narrative Structure**:
     - *Ordering*: Linear and Random Access
     - *Interactivity*: Filtering/Selection/Search, Navigation Buttons
     - *Messaging*: Captions/Headlines, Annotataions
     
*Combining linear and random access,not only can be tell our story, but users are able to have more freedom to choose what they are interested in. The figure shown below gives an example of interactivity and messaging. Users can find more patterns by choosing a specific period using the filter on the right to. The navigation buttons on the top help users find the complaint type temporal patterns efficiently.**

 ![Drag Racing](overall2.png)

***
## **Ⅴ. Visualizations**
### **Explain the visualizations you've chosen.**

### **Why are they right for the story you want to tell?**

### The overall storyline
*It begins with the visualization of COVID-19, which follows with an overview of the annual regularity for the main complaint types during the period from May 2015 to May 2020. Then we explore the spatial and spatiotemporal regularities for complaint types, and correlation with COVID-19 in New York city by using machine learning methods. In the end, the complaint types and the corresponding response departments are investigated to provide suggestions to help improve the efficiency.*

### ***1. For the COVID-19 visualization:*** 
#### **We choose time domain bar chart for the daily case count dataset. We create an interactive chart so that the reader can make parallel comparison of cases, hospitalized and death counts. It is clear for user to know exactly the trend.**

In [12]:
df_case = pd.read_csv('coronavirus-data-master/case-death.csv')
output_notebook(hide_banner=True)
daterange = (df_case['DATE_OF_INTEREST'])
color = ['#1f77b4','#aec7e8','#ff7f0e']
source = ColumnDataSource(df_case)
p = figure(x_range = FactorRange(factors=daterange),title="Cases of Covid-19",
           plot_width=1300, plot_height = 650, y_range=(0, 6500), x_axis_label='Date',y_axis_label='Cases')
bar ={}
items = []
for (indx,i),col in zip(enumerate(df_case.iloc[:,1:]),Spectral4[1:]):
    bar[i] = p.vbar(x= 'DATE_OF_INTEREST', top=i, source= source, width = 0.9,muted_alpha=0.5, muted=True,color=col, fill_alpha = 0.6) 
    
    items.append((i, [bar[i]]))
legend = Legend(items= items , location=(0, -20),click_policy="hide",label_text_font_size="12pt" )
p.add_layout(legend, 'right')
#output_file("interactive_legend.html", title="interactive_legend.py example")
p.xaxis.major_label_orientation = math.pi/4
show(p)

#### *From the visualization above, we can see a trend of ascending initially and then descending. However, the descending period is not reliable, since the official website of NYC has declared that data needs correction, sepecially recent data, which makes recent data trustless.*
## 

#### **For rate-by-age, rate-by-sex, rate-by-borough datasets, we also created interactive parallel comparison charts, because it is an efficient way to provide more options for viewer to discover valuable information.**

In [13]:
df_age = pd.read_csv('coronavirus-data-master/by-age.csv')
output_notebook(hide_banner=True)
source = ColumnDataSource(df_age)
p = figure(x_range = FactorRange(factors = df_age['AGE_GROUP']),title="Rates by Age ",
           plot_width=1200, plot_height = 600, y_range = (0, 4100), x_axis_label='Age Group', y_axis_label= 'Rate per 100,000 people')
bar ={}
items = []
for (indx,i),col in zip(enumerate(df_age.iloc[:,1:]),Spectral4[1:]):
    bar[i] = p.vbar(x= 'AGE_GROUP', top=i, source= source, width = 0.65,muted_alpha=0.5, muted=True,color=col, fill_alpha = 0.6) 
    
    items.append((i, [bar[i]]))
legend = Legend(items= items , location=(0, -20),click_policy="hide",label_text_font_size= '12pt')#"8pt"
p.add_layout(legend, 'right')
show(p)

#### *This plot explicitly illustrates the risks people of different age group suffers. As we can see, old people over 65 years old are high risk groups, and the risk increases correspindingly with the increase of age. Teenagers and children under 17 years old are less susceptible to the infection of COVID-19*
# 

In [14]:
df_sex = pd.read_csv('coronavirus-data-master/by-sex.csv')
output_notebook(hide_banner=True)
source = ColumnDataSource(df_sex)
p = figure(x_range = FactorRange(factors = df_sex['SEX_GROUP']),title="Rates by Sex ",
           plot_width=900, plot_height = 450, y_range = (0, 4100), x_axis_label='Sex', y_axis_label= 'Rate per 100,000 people')
bar ={}
items = []
for (indx,i),col in zip(enumerate(df_sex.iloc[:,1:]),Spectral4[1:]):
    bar[i] = p.vbar(x= 'SEX_GROUP', top=i, source= source, width = 0.4,muted_alpha=0.5, muted=True,color=col, fill_alpha = 0.6) 
    
    items.append((i, [bar[i]]))
legend = Legend(items= items , location=(0, -20),click_policy="hide",label_text_font_size= '9pt')#"8pt"
p.add_layout(legend, 'right')
show(p)

#### *According to the plot, males' case rate, hospitalized rate and death rate are slightly higher than females'.*
# 

In [15]:
df_borough = pd.read_csv('coronavirus-data-master/boro.csv')
df_borough['COVID_CASE_RATE'] = df_borough['COVID_CASE_RATE']*10
df_borough = df_borough.iloc[:-1,:]
output_notebook(hide_banner=True)
color = ['#1f77b4','#aec7e8','#ff7f0e']
source = ColumnDataSource(df_borough)
p = figure(x_range = FactorRange(factors = df_borough['BOROUGH_GROUP']),title="Borough Statistics",
           plot_width=900, plot_height = 450, y_range = (0, 65000), x_axis_label='Borough', y_axis_label= 'Counts & Rates');
bar ={}
items = []
for (indx,i),col in zip(enumerate(df_borough.iloc[:,1:]),color[1:]):#[Spectral4[2], Spectral4[0]]
    bar[i] = p.vbar(x= 'BOROUGH_GROUP', top=i, source= source, width = 0.4,muted_alpha=0.6, muted=True,color=col, fill_alpha = 0.6) 
    
    items.append((i, [bar[i]]))
legend = Legend(items= items , location=(0, -20),click_policy="hide",label_text_font_size= '9pt')#"8pt"
p.add_layout(legend, 'right')
show(p)

### *The unit of 'COVID_CASE_RATE'  is  'rate per 1,000,000 people'. What's worth mentioning is that although Stanten Island have the least case amount, its case rate is quite high. Probably because it's a relatively small borough and the population density is high, which makes the virus spread easier.*

***
***2. For the annual regularity exploration:*** 
*We choose time domain line chart. We create an interactive chart so that the reader can make parallel comparison of any selected years. Using this parallel comparison line chart, not only can the user detect annually regularities of a certain type, but they can explore the difference between years.*

This part is quite time-consuming, so we provide the codes, as well as the results(imported as HTML file).

In [ ]:
## This part is quite time-consuming, so we provide the codes(annotated, if you want to run it please remove the comment symbol), as well as the result(imported as HTML).
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import warnings
from IPython.display import HTML
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook, show, output_file, save
from bokeh.models import ColumnDataSource,FactorRange,Legend
from bokeh.palettes import Category20
from bokeh.plotting import figure
from bokeh.transform import factor_cmap,dodge
from bokeh.palettes import Spectral4
from bokeh.models import Panel
from bokeh.models.widgets import Tabs
import math

## defining a average-smooth filter, which can make the annual line chart much more smooth, so that we can focus on the general trend.
def moving_average(interval, window_size):
    window = np.ones(int(window_size)) / float(window_size)
    return np.convolve(interval, window, 'valid')


# read in data and data arrangement
df = pd.read_csv('focus_types.csv')
df['Created Date'] = pd.to_datetime(df['Created Date'], format = '%Y-%m-%d')
df['Closed Date'] = pd.to_datetime(df['Closed Date'], format = '%Y-%m-%d')
df =df.loc[df['Created Date'].dt.date >= datetime.date(2015,1,1)]
df['Count'] = np.ones([len(df),1])

group_type = df.groupby('Complaint Type').sum()
ctype = group_type.index    #get all the type of requests

## Total daily amount
output_notebook(hide_banner=True)
daterange = list(map(str,np.arange(0,367)))
#print(daterange)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#1f77b4','#17becf','#ff7f0e','#2ca02c','#9467bd']
p = figure(title="Amount of Daily Total Complaint",#x_axis_type='datetime',#x_range = FactorRange(factors=daterange),
           plot_width=950, plot_height = 600,  x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')
#y_range=(0, 6500)
for year, color in zip([2015, 2016,2017,2018,2019],color):#
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1))]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')#
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
p.xaxis.visible = True
tab0 = Panel(child=p, title="All Types")
output_file('Total Case amount.html')
show(p)


## 'Rodent'
output_notebook(hide_banner=True)
#daterange = list(map(str,np.arange(1,367)))
#print(daterange)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'Rodent'", #x_range = FactorRange(factors=daterange),
           plot_width=950, plot_height = 600,  x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')
#y_range=(0, 6500)
for year, color in zip([2015,2016, 2017, 2018, 2019],color):#
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'Rodent')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
tab1 = Panel(child=p, title="Rodent")
output_file('Rodent.html')
show(p)


# UNSANITARY CONDITION
output_notebook(hide_banner=True)
#print(daterange)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'UNSANITARY CONDITION'", #,x_range = FactorRange(factors=daterange),
           plot_width=950, plot_height = 600,  x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')
        #y_range=(0, 6500)
for year, color in zip([2015,2016, 2017, 2018, 2019],color):#
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'UNSANITARY CONDITION')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_left"
p.legend.click_policy="hide"
tab2 = Panel(child=p, title="UNSANITARY CONDITION")
output_file("UNSANITARY CONDITION.html")
show(p)


# DIRTY CONDITION
output_notebook(hide_banner=True)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'Dirty Conditions'",
           plot_width=950, plot_height = 600,  x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')
#y_range=(0, 6500)
for year, color in zip([2015,2016, 2017, 2018, 2019],color):#
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'Dirty Conditions')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
tab3 = Panel(child=p, title="DIRTY CONDITION")
output_file("DIRTY CONDITION.html")
show(p)


# Noise-Residential
output_notebook(hide_banner=True)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'Noise - Residential'",
           plot_width=950, plot_height = 600, y_range=(370, 1200), x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')

for year, color in zip([2015,2016, 2017, 2018, 2019],color):
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'Noise - Residential')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
tab4 = Panel(child=p, title='Noise-Residential')
output_file("Noise-Residential.html")
show(p)


# Noise-Commercial
output_notebook(hide_banner=True)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'Noise - Commercial'",
           plot_width=950, plot_height = 600, 
           y_range=(0, 235), 
           x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')

for year, color in zip([2015,2016, 2017, 2018, 2019],color):#
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'Noise - Commercial')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
tab5 = Panel(child=p, title='Noise-Commercial')
output_file("Noise-Commercial.html")
show(p)


# Noise-Street_Sidewalk
output_notebook(hide_banner=True)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'Noise - Street/Sidewalk'",
           plot_width=950, plot_height = 600, 
           #y_range=(0, 1400), 
           x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')

for year, color in zip([2015,2016, 2017, 2018, 2019],color):
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'Noise - Street/Sidewalk')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
tab6 = Panel(child=p, title='Noise-Street/Sidewalk')
output_file("Noise-Street_Sidewalk.html")
show(p)


# Water System
output_notebook(hide_banner=True)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'Water System'",
           plot_width=950, plot_height = 600, 
           #y_range=(0, 1400), 
           x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')

for year, color in zip([2015,2016, 2017, 2018, 2019],color):
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'Water System')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
tab7 = Panel(child=p, title='Water System')
output_file("Water System.html")
show(p)


# HEAT-HOT WATER
output_notebook(hide_banner=True)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'HEAT/HOT WATER'",
           plot_width=950, plot_height = 600, 
           #y_range=(0, 1400), 
           x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')

for year, color in zip([2015,2016, 2017, 2018, 2019],color):
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'HEAT/HOT WATER')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
p.xaxis.visible = True
tab8 = Panel(child=p, title='HEAT/HOT WATER')
output_file("HEAT-HOT WATER.html")
show(p)


# Blocked Driveway
output_notebook(hide_banner=True)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'Blocked Driveway'",
           plot_width=950, plot_height = 600, 
           y_range=(0, 650), 
           x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')

for year, color in zip([2015,2016, 2017, 2018, 2019],color):
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'Blocked Driveway')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
tab9 = Panel(child=p, title='Blocked Driveway')
output_file("Blocked Driveway.html")
show(p)


# Building-Use
output_notebook(hide_banner=True)
#color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78',,'#98df8a','#d62728','#ff9896',,'#c5b0d5','#e377c2','#f7b6d2','#9edae5']
color = ['#17becf','#ff7f0e','#2ca02c','#9467bd','#1f77b4']
p = figure(title="Amount of Daily Total Complaint of 'Building/Use'",
           plot_width=950, plot_height = 600, 
           y_range=(0, 150), 
           x_axis_label='Day of Year (month larger than 12 means the coming month of next year)',y_axis_label='Case amount')

for year, color in zip([2015, 2016, 2017, 2018, 2019],color):#
    #print(year)
    df_inter1 = df[(df['Created Date'].dt.date >= datetime.date(year,5,1)) & (df['Created Date'].dt.date < datetime.date(year+1,5,1)) & (df['Complaint Type'] == 'Building/Use')]
    df_inter1['Created Date'] = df_inter1['Created Date'].dt.date
    group_temp = df_inter1.groupby('Created Date').sum()
    p.line(group_temp.index.map(lambda x: (x.month + 12*(x.year - year))+(x.day/31)), moving_average(group_temp['Count'],20), line_width=2, color=color, alpha=0.8, legend_label=str(year)+'.05 — '+str(year+1)+'.05')
    del df_inter1,group_temp
p.legend.location = "top_right"
p.legend.click_policy="hide"
tab10 = Panel(child=p, title='Building/Use')
output_file("Building-Use.html")
show(p)


# after computing tab0-tab10, we are now gathering them to produce an intergrated interactive chart.
output_file("overview.html")
t = Tabs(tabs = [tab0, tab1, tab2, tab3, tab4, tab5, tab6, tab7, tab8, tab9, tab10])
show(t)

#### ***The code below load in the result of the module above*** 

In [9]:
from IPython.display import HTML
from IPython.display import IFrame
IFrame(src="overview.html", width=900, height=600)
# HTML(filename="overview.html")

***
***3. Spatiotemporal patterns exploration:*** 
*We choose four typical complaint types, and use heat map movies to find time and space patterns. Not only can the movies illustrate spatiotemporal patterns, but they are able to show the influence of COVID-19 on differnt complaint types which could represent the change of people's lives.*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
import folium
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource,FactorRange,Legend
from bokeh.palettes import Paired12
from bokeh.plotting import figure
from bokeh.transform import factor_cmap,dodge
import math
from bokeh.layouts import column, row, WidgetBox
from bokeh.models import Panel
from bokeh.models.widgets import Tabs
from bokeh.models.widgets import CheckboxGroup
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider
from bokeh.models.widgets import Dropdown, Toggle
from bokeh.layouts import layout
from folium.plugins import HeatMap
from folium import plugins
from sklearn.neighbors import KNeighborsClassifier
from jinja2 import Template
from folium.map import Layer
data = pd.read_csv('focus_types.csv')
data['Created Date'] = pd.to_datetime(data['Created Date'],format = '%Y-%m-%d %H:%M:%S')
data_st = data.sort_values(by = ['Created Date'])
data_pre = data_st.loc[data_st['Created Date'].dt.date >= datetime.date(2015,1,1),:]
data_pre = data_pre.loc[data_pre['Created Date'].dt.date < datetime.date(2019,1,1),:]
data_pre['Closed Date'] = pd.to_datetime(data_pre['Closed Date'],format = '%Y-%m-%d %H:%M:%S')

data_pre['DayOfWeek_s'] = data_pre['Created Date'].dt.weekday
# data_pre['DayOfWeek_s'].replace([0, 1, 2, 3, 4, 5, 6],['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], inplace = True) 
data_pre['Hour_s'] = data_pre['Created Date'].dt.hour
data_pre['Month_s'] = data_pre['Created Date'].dt.month
data_pre = data_pre.sample(n=int((len(data_pre))/5))

Rodent_random = data_pre[data_pre['Complaint Type'] == 'Rodent']
NoiseCom_random = data_pre[data_pre['Complaint Type'] == 'Noise - Commercial']
US_random = data_pre[data_pre['Complaint Type'] == 'UNSANITARY CONDITION']
DirtyCondition_random = data_pre[data_pre['Complaint Type'] == 'Dirty Conditions']
BlockDrive_random = data_pre[data_pre['Complaint Type'] == 'Blocked Driveway']
NoiseResi_random = data_pre[data_pre['Complaint Type'] == 'Noise - Residential']


Rodent_random.dropna(inplace = True)
NoiseCom_random.dropna(inplace = True)
BlockDrive_random.dropna(inplace = True)
NoiseResi_random.dropna(inplace = True)

XY=[]
for i in range(len(Rodent_random)):
    XY.append([float(Rodent_random['Location'].values[i].split()[0][1:-1]),float(Rodent_random['Location'].values[i].split()[1][0:-1])])
Rodent_random['XY']=XY
XY =[]

for i in range(len(NoiseCom_random)):
    XY.append([float(NoiseCom_random['Location'].values[i].split()[0][1:-1]),float(NoiseCom_random['Location'].values[i].split()[1][0:-1])])
NoiseCom_random['XY']=XY
XY =[]


for i in range(len(BlockDrive_random)):
    XY.append([float(BlockDrive_random['Location'].values[i].split()[0][1:-1]),float(BlockDrive_random['Location'].values[i].split()[1][0:-1])])
BlockDrive_random['XY']=XY
XY = []

for i in range(len(NoiseResi_random)):
    XY.append([float(NoiseResi_random['Location'].values[i].split()[0][1:-1]),float(NoiseResi_random['Location'].values[i].split()[1][0:-1])])
NoiseResi_random['XY']=XY
XY = []

#set hour
H = Rodent_random['Created Date'].dt.hour.copy()
Rodent_random['hour'] = H.copy()

H = NoiseResi_random['Created Date'].dt.hour.copy()
NoiseResi_random['hour'] = H.copy()

H = NoiseCom_random['Created Date'].dt.hour.copy()
NoiseCom_random['hour'] = H.copy()

H = BlockDrive_random['Created Date'].dt.hour.copy()
BlockDrive_random['hour'] = H.copy()
####
heat_time1 = [[row['XY'] for index, row in Rodent_random[Rodent_random['hour'] == i].iterrows()] for i in range(0,23)]
# hm1 = plugins.HeatMapWithTime(heat_time1,auto_play=True,min_opacity=0.2,overlay=False)
# hm1.add_to(map_NY)

heat_time2 = [[row['XY'] for index, row in NoiseResi_random[NoiseResi_random['hour'] == i].iterrows()] for i in range(0,23)]


heat_time3= [[row['XY'] for index, row in NoiseCom_random[NoiseCom_random['hour'] == i].iterrows()] for i in range(0,23)]

heat_time4=[[row['XY'] for index, row in BlockDrive_random[BlockDrive_random['hour'] == i].iterrows()] for i in range(0,23)]

class HeatMapWithTimeAdditional(Layer):
    _template = Template("""
        {% macro script(this, kwargs) %}
            var {{this.get_name()}} = new TDHeatmap({{ this.data }},
                {heatmapOptions: {
                    radius: {{this.radius}},
                    minOpacity: {{this.min_opacity}},
                    maxOpacity: {{this.max_opacity}},
                    scaleRadius: {{this.scale_radius}},
                    useLocalExtrema: {{this.use_local_extrema}},
                    defaultWeight: 1,
                    {% if this.gradient %}gradient: {{ this.gradient }}{% endif %}
                }
            }).addTo({{ this._parent.get_name() }});
        {% endmacro %}
    """)

    def __init__(self, data, name=None, radius=15,
                 min_opacity=0, max_opacity=0.6,
                 scale_radius=False, gradient=None, use_local_extrema=False,
                 overlay=True, control=True, show=True):
        super(HeatMapWithTimeAdditional, self).__init__(
            name=name, overlay=overlay, control=control, show=show
        )
        self._name = 'HeatMap'
        self.data = data

        # Heatmap settings.
        self.radius = radius
        self.min_opacity = min_opacity
        self.max_opacity = max_opacity
        self.scale_radius = 'true' if scale_radius else 'false'
        self.use_local_extrema = 'true' if use_local_extrema else 'false'
        self.gradient = gradient
        
heatmap = folium.Map(location=[40.693943, -73.985880], zoom_start=11, height=600)
plugins.HeatMapWithTime(heat_time1,overlay=True,name='data1',min_opacity=0.05,min_speed=0.1,speed_step=0.1,max_speed=1,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time2,overlay=True,name='data2',min_opacity=0.05,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time3,overlay=True,name='data3',min_opacity=0.05,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time4,overlay=True,name='data4',min_opacity=0.05,max_opacity=1).add_to(heatmap)
heatmap.add_child(folium.LayerControl())
heatmap

# weeks
heat_time1 = [[row['XY'] for index, row in Rodent_random[Rodent_random['DayOfWeek_s'] == i].iterrows()] for i in range(0,7)]
heat_time2 = [[row['XY'] for index, row in NoiseResi_random[NoiseResi_random['DayOfWeek_s'] == i].iterrows()] for i in range(0,7)]
heat_time3= [[row['XY'] for index, row in NoiseCom_random[NoiseCom_random['DayOfWeek_s'] == i].iterrows()] for i in range(0,7)]
heat_time4=[[row['XY'] for index, row in BlockDrive_random[BlockDrive_random['DayOfWeek_s'] == i].iterrows()] for i in range(0,7)]

heatmap = folium.Map(location=[40.693943, -73.985880], zoom_start=11, height=600)
plugins.HeatMapWithTime(heat_time1,overlay=True,name='data1',min_opacity=0.05,min_speed=0.1,speed_step=0.1,max_speed=1,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time2,overlay=True,name='data2',min_opacity=0.05,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time3,overlay=True,name='data3',min_opacity=0.05,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time4,overlay=True,name='data4',min_opacity=0.05,max_opacity=1).add_to(heatmap)
heatmap.add_child(folium.LayerControl())
heatmap

#month
heat_time1 = [[row['XY'] for index, row in Rodent_random[Rodent_random['Month_s'] == i].iterrows()] for i in range(0,11)]
heat_time2 = [[row['XY'] for index, row in NoiseResi_random[NoiseResi_random['Month_s'] == i].iterrows()] for i in range(0,11)]
heat_time3= [[row['XY'] for index, row in NoiseCom_random[NoiseCom_random['Month_s'] == i].iterrows()] for i in range(0,11)]
heat_time4=[[row['XY'] for index, row in BlockDrive_random[BlockDrive_random['Month_s'] == i].iterrows()] for i in range(0,11)]

heatmap = folium.Map(location=[40.693943, -73.985880], zoom_start=11, height=600)
plugins.HeatMapWithTime(heat_time1,overlay=True,name='data1',min_opacity=0.05,min_speed=0.1,speed_step=0.1,max_speed=1,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time2,overlay=True,name='data2',min_opacity=0.05,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time3,overlay=True,name='data3',min_opacity=0.05,max_opacity=1).add_to(heatmap)
HeatMapWithTimeAdditional(heat_time4,overlay=True,name='data4',min_opacity=0.05,max_opacity=1).add_to(heatmap)
heatmap.add_child(folium.LayerControl())
heatmap

Since the code is time-consuming, the results could be found in temporal-spatial regularity part of the website version.
Below gives an example of the six figures.

In [8]:
IFrame(src="hours_afterCovid0.html", width=900, height=600)

***
***4. Statistics by the department and what they are responsible for:*** 
*To see which department is the most important one and see how it allocated for different request.*

In [ ]:
data = pd.read_csv("focus_types.csv")
data['Created Date'] = pd.to_datetime(data['Created Date'],format = '%Y-%m-%d %H:%M:%S')
data_st = data.sort_values(by = ['Created Date'])
data_pre = data_st.loc[data_st['Created Date'].dt.date >= datetime.date(2015,1,1),:]
data_pre = data_pre.loc[data_pre['Created Date'].dt.date < datetime.date(2019,1,1),:]
data_pre['Closed Date'] = pd.to_datetime(data_pre['Closed Date'],format = '%Y-%m-%d %H:%M:%S')
Agency_type = data_pre.loc[:, ['Agency Name']]
Agency_type_sum = Agency_type.groupby('Agency Name').size()
Agency_type_sum_order =  Agency_type_sum.sort_values(ascending=False)
AgencyNames = Agency_type_sum_order._stat_axis.values[0:12]
Agency_type_sum_order_value=Agency_type_sum_order.values[0:12]
plt.figure(figsize=(9,5))
plt.bar(AgencyNames,Agency_type_sum_order_value)
plt.xticks(rotation=90)
plt.ylabel('Number of service request',fontsize = 12)
plt.show()

Agency_type = pd.crosstab(data_pre['Complaint Type'],data_pre['Agency Name'])
for i in range(len(Agency_type.columns)):
    num = Agency_type.loc[:,Agency_type.columns[i]].sum()
    for j in range(len(Agency_type.index)):
        Agency_type.iloc[j,i] = Agency_type.iloc[j,i]/num    
        
Agency_type = Agency_type.loc[:,AgencyNames]
Agency_type = Agency_type.reset_index()

source = ColumnDataSource(Agency_type)
color = Paired12
c_type = data_pre.groupby(['Complaint Type']).count().index.values
output_notebook()
p = figure(x_range = FactorRange(factors=c_type),plot_width=1000,x_axis_label='Complaint type',y_axis_label='Relative Frequency')
p.xaxis.major_label_orientation = math.pi/2

bar ={}
items = [] ### for the custom legend // you need to figure out where to add it
for (indx,i),col in zip(enumerate(AgencyNames),color):
    bar[i] = p.vbar(x=dodge('Complaint Type',0, range=p.x_range), top=i, source= source, width = 0.9,muted_alpha=0.03, muted=True,color=col, alpha = 0.5) 
    items.append((i, [bar[i]]))
    
legend = Legend(items= items , location=(0, -20),click_policy="mute",label_text_font_size="8pt" )
p.add_layout(legend, 'right')
output_file()
show(p)

***
***5. Explore how are the different types of requests distributed in different time pattern:*** 
*To see which department is the most important one and see how it allocated for different request. Here we put three interactive graph togather, which will be more clear*

In [ ]:
focuscrimes = data_pre.groupby(['Complaint Type']).count().index.values
data_pre['DayOfWeek_s'] = data_pre['Created Date'].dt.weekday
data_pre['DayOfWeek_s'].replace([0, 1, 2, 3, 4, 5, 6],['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], inplace = True) 
data_pre['Hour_s'] = data_pre['Created Date'].dt.hour
data_pre['Month_s'] = data_pre['Created Date'].dt.month

hour_type = pd.crosstab(data_pre['Hour_s'],data_pre['Complaint Type'])
for i in range(len(hour_type.columns)):
    num = hour_type.loc[:,hour_type.columns[i]].sum()
    for j in range(len(hour_type.index)):
        hour_type.iloc[j,i] = hour_type.iloc[j,i]/num    
        
hour_type = hour_type.loc[:,focuscrimes]
hour_type = hour_type.reset_index()

week_type = pd.crosstab(data_pre['DayOfWeek_s'],data_pre['Complaint Type'])
for i in range(len(week_type.columns)):
    num = week_type.loc[:,week_type.columns[i]].sum()
    for j in range(len(week_type.index)):
        week_type.iloc[j,i] = week_type.iloc[j,i]/num    
        
week_type = week_type.loc[:,focuscrimes]
week_type = week_type.reset_index()

month_type = pd.crosstab(data_pre['Month_s'],data_pre['Complaint Type'])
for i in range(len(month_type.columns)):
    num = month_type.loc[:,month_type.columns[i]].sum()
    for j in range(len(month_type.index)):
        month_type.iloc[j,i] = month_type.iloc[j,i]/num    
        
month_type = month_type.loc[:,focuscrimes]
month_type = month_type.reset_index()

source_hour = ColumnDataSource(hour_type)
source_week = ColumnDataSource(week_type)
source_month = ColumnDataSource(month_type)

color = color = ['#1f77b4','#aec7e8','#ff7f0e','#ffbb78','#2ca02c','#98df8a','#d62728','#ff9896','#9467bd','#c5b0d5','#e377c2','#f7b6d2','#17becf','#9edae5']
weekdays = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
output_notebook()

p1 = figure(x_range = FactorRange(factors=weekdays),plot_width=1000,x_axis_label='Weekday',y_axis_label='Relative Frequency')

bar ={}
items = [] ### for the custom legend // you need to figure out where to add it
for (indx,i),col in zip(enumerate(focuscrimes),color):
    bar[i] = p1.vbar(x=dodge('DayOfWeek_s',0, range=p1.x_range), top=i, source= source_week, width = 0.5,muted_alpha=0.005, muted=True,color=col,alpha=0.5) 
    items.append((i, [bar[i]]))

legend = Legend(items= items , location=(0, -20),click_policy="mute",label_text_font_size="8pt" )
p1.add_layout(legend, 'right')

tab1 = Panel(child=p1, title="Weekly")

hours = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
p2 = figure(x_range = FactorRange(factors=hours),plot_width=1000,x_axis_label='Hour',y_axis_label='Relative Frequency')

bar ={}
items = [] ### for the custom legend // you need to figure out where to add it
for (indx,i),col in zip(enumerate(focuscrimes),color):
    bar[i] = p2.vbar(x=dodge('Hour_s',0.5, range=p2.x_range), top=i, source= source_hour, width = 0.5,muted_alpha=0.005, muted=True,color=col,alpha=0.5) 
    items.append((i, [bar[i]]))

legend = Legend(items= items , location=(0, -20),click_policy="mute",label_text_font_size="8pt" )
p2.add_layout(legend, 'right')

tab2 = Panel(child=p2, title="Hourly")


months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
p3 = figure(x_range = FactorRange(factors=months),plot_width=1000,x_axis_label='Month',y_axis_label='Relative Frequency')

bar ={}
items = [] ### for the custom legend // you need to figure out where to add it
for (indx,i),col in zip(enumerate(focuscrimes),color):
    bar[i] = p3.vbar(x=dodge('Month_s',-0.5, range=p3.x_range), top=i, source= source_month, width = 0.5,muted_alpha=0.005, muted=True,color=col,alpha=0.5) 
    items.append((i, [bar[i]]))

legend = Legend(items= items , location=(0, -20),click_policy="mute",label_text_font_size="8pt" )
p3.add_layout(legend, 'right')

tab3 = Panel(child=p3, title="Monthly")

tabs = Tabs(tabs=[tab2, tab1, tab3])
output_file('time.html')
show(tabs)

***
***6. How the relation between the created and closed time and the complaint request:*** 
*We creat the boxplot to see the basic statistical value of the respond time*

In [ ]:
data_pre['Respond time'] = data_pre['Closed Date'] - data_pre['Created Date']
day = pd.DatetimeIndex(data_pre['Respond time'].values).day
hour = pd.DatetimeIndex(data_pre['Respond time'].values).hour
minute = pd.DatetimeIndex(data_pre['Respond time'].values).minute
second = pd.DatetimeIndex(data_pre['Respond time'].values).second
time = (day - 1)*24+hour+minute/60+second/3600
data_pre['Respond time'] = time

valid_resptime = data_pre.loc[data_pre['Respond time'] > 0]
valid_resptime.index = valid_resptime['Complaint Type']
valid_resptime_foc = valid_resptime.loc[focuscrimes,:]
focusdata_rt = valid_resptime_foc.loc[:,['Complaint Type', 'Respond time']]
focusdata_rt = focusdata_rt.reset_index(drop=True)
focusdata_rt.boxplot("Respond time", by="Complaint Type",figsize = (10,5), rot=90, fontsize=10)
plt.ylabel('Hour')
plt.show()

***
***4. The distribution in the map:*** 
*We use folium to build the map and add the point for one certain type of complaint to see the distribution. Also we gennerate a 50·50 grid and use KNN to clasify which type should one be*

This part is quite time-consuming, so we provide the codes, as well as the results(imported as HTML file), and only when the K_neighbor = 10.

In [ ]:
data_random = data_pre.sample(frac=1/40)
data_random.dropna(inplace = True)
NR = data_random[(data_random['Complaint Type'] == 'Noise - Residential')]

NR_FG = folium.FeatureGroup(name = "Noise - Residential")

for index, row in NR.iterrows():
    folium.CircleMarker([float(row['Location'].split()[0][1:-1]),float(row['Location'].split()[1][0:-1])],
                      radius = 1,
                      color = 'brown'
                        ).add_to(NR_FG)
    
NYCmapFeature = folium.Map([40.693943, -73.985880], zoom_start = 11)
NYCmapFeature.add_child(NR_FG)
display(NYCmapFeature)

NR_random = data_pre[data_pre['Complaint Type'] == 'Noise - Residential'].sample(n=100000)
BD_random = data_pre[data_pre['Complaint Type'] == 'Blocked Driveway'].sample(n=100000)
US_random = data_pre[data_pre['Complaint Type'] == 'UNSANITARY CONDITION'].sample(n=100000)
sample_type = pd.concat([NR_random, BD_random, US_random])
sample_type.dropna(inplace = True)

n_neighbors = [5,10,15]
X = []
for i in range(len(sample_type)):
    X.append([float(sample_type['Location'].values[i].split()[0][1:-1]),float(sample_type['Location'].values[i].split()[1][0:-1])])
y = sample_type['Complaint Type']

cla_map_hooray=map_hooray
cla = folium.FeatureGroup(name = 'clasification')
for k in n_neighbors:
    for i in np.arange(-0.12,0.24,0.0072):
        for j in np.arange(-0.06,0.26,0.0064):
            clf = KNeighborsClassifier(k)
            x = [40.693943+i, -73.985880+j]
            clf.fit(X, y)
            predicted_complaint = clf.predict([x])
            if predicted_complaint == 'Noise - Residential':
                    folium.CircleMarker([40.693943+i, -73.985880+j],
                            radius=0.5,
                            color='green',
                           ).add_to(cla)
            if predicted_complaint == 'Blocked Driveway':
                    folium.CircleMarker([40.693943+i, -73.985880+j],
                            radius=0.5,
                            color='red',
                           ).add_to(cla)            
            if predicted_complaint == 'UNSANITARY CONDITION':
                    folium.CircleMarker([40.693943+i, -73.985880+j],
                            radius=0.5,
                            color='blue',
                           ).add_to(cla)           
                   
    #Create map and add feature group
    mapFeature=folium.Map([40.693943, -73.985880], zoom_start=11)
    mapFeature.add_child(cla)
    display(mapFeature)

In [10]:
IFrame(src="kneighbors2v10.html", width=1000, height=600)

***
## __Ⅵ. Discussion__ _Think critically about your creation_
### **1. What went well?**
*We tried our best to make the interactive graph so that it will be easier for the reader to find the content that they are interested, and also in this way, it would be tidier in the whole without some repetitive graphs. Furthermore, we combined some same type of interactive graphs together, e.g. different time pattern, so that it would be more clear.*

*We did not only focus on the 311 complaints request dataset, but also the COVID - 19 dataset. During our data analysis, we splited our 311 complaint request dataset into 2 part: "before and after COVID - 19 appeared", comparing and discussing the same and the different rule and see how the COVID - 19 affect the complaint request.*

### **2. What is still missing? What could be improved? Why?**
*In the KNN part, we only adapted a square area to predict the type of complaint, which seems not precise enough. To be more specific, the area contains some water area and the area outside NYC. It would be better if we built an area that has the shape of NYC.*

*We only implemented KNN for machine learning related part, and we planned to use regression model, for example, using the complaint data to predict the trend of COVID - 19. However, we were limited with the information of COVID - 19 related data and also the 311 complaint data is not updated (The official claims that they will update as soon)¶*

***
## __Ⅵ. Contributions__ _Who did what?_
### **You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That's what you should explain).It is not OK simply to write "All group members contributed equally".**
***

+ *Xiaoran Ma is mainly responsible for visualization of CVOID-19 and overall temporal patterns for complaint type.*
+ *Yunpeng Jiang is mainly responsible for the analysis of the data and spatial patterns using machine learning, and visualize the complaints type in different time patterns*
+ *Chuan Wen is responsible for spatiotemporal patterns analysis using heatmap movies and writing storyline.*